In [ ]:
# import libraries
!pip install tensorflow
# tensorflow_io 0.28 is compatible with TensorFlow 2.11
!pip install tensorflow_io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 46.0 MB/s eta 0:00:00


In [ ]:
# import libaries
import io
import re
import string
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import random
import os
import torch
#import pacakges
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
#import tensorflow_io as tfio

In [ ]:
# Build the audio model
import tensorflow as tf
!pip install transformers==4.28.0
from transformers import TFAutoModel

# Load the pre-trained model using the Hugging Face interface
model_checkpoint = "facebook/wav2vec2-base"
huggingface_model = TFAutoModel.from_pretrained(model_checkpoint, trainable=False, from_pt=True)

# Define the inputs to the model
input_values = tf.keras.Input(shape=(16000,), dtype=tf.float32)

# Pass the inputs through the Wav2Vec model
wav2vec_output = huggingface_model(input_values)

# Create the TensorFlow functional API model
audio_model = tf.keras.Model(inputs=input_values, outputs=wav2vec_output)

# Print the model summary
audio_model.summary()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.2 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(



TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tine this model, you need a GPU or a TPU
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFWav2Vec2Model: ['project_hid.weight', 'project_q.weight', 'project_hid.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing TFWav2Vec2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFWav2Vec2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the che

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16000)]           0         
                                                                 
 tf_wav2_vec2_model (TFWav2V  TFWav2Vec2BaseModelOutpu  94371712 
 ec2Model)                   t(last_hidden_state=(Non            
                             e, 49, 768),                        
                              extract_features=(None,            
                              49, 512),                          
                              hidden_states=None, att            
                             entions=None)                       
                                                                 
Total params: 94,371,712
Trainable params: 0
Non-trainable params: 94,371,712
_________________________________________________________________


In [ ]:
from keras.models import Sequential

In [ ]:
# Load pretrained word2vec embeddings
from gensim.models import KeyedVectors
word2vec_vectors = KeyedVectors.load("/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/word2vec_embeddings/word2vec.wordvectors", mmap='r')

In [ ]:
# Load the vocabulary dictionary
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/final_combined_data_original_augmented/vocab_dict.pkl', 'rb') as handle:
    data = handle.read()
vocab = pickle.loads(data)

In [ ]:
# Get the word embeddings for each word
#vocab = tokenizer.word_index
from gensim.models.keyedvectors import Word2VecKeyedVectors
import numpy as np
def get_weight_matrix():
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((len(vocab)+1, word2vec_vectors.vector_size))
    i=0
    for key in vocab.keys():
      if key=='OOV':
        continue
      elif key not in word2vec_vectors:
        i=i+1
        continue
      else:
        weight_matrix[i + 1] = word2vec_vectors[key]
        i=i+1
    return weight_matrix

In [ ]:
embedding_vectors = get_weight_matrix()

In [ ]:
# Create the embedding layer
embedding_layer = Embedding(input_dim=len(vocab) + 1,
                                output_dim=300,
                                weights=[embedding_vectors],
                                input_length=50,
                                trainable=False)

In [ ]:
# Build word and text models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Concatenate, Reshape, LSTM, Dense

# Define input layers
word_input = Input(shape=(50))


# Define embedding layer

# Embed word and pos inputs
word_embedded = embedding_layer(word_input)


# Reshape the concatenated tensor
#reshaped = Reshape((-1, 16))(concatenated)

# Apply LSTM layer
lstm_output = LSTM(16, dropout=0.2, recurrent_dropout=0.2)(word_embedded)

# Apply dense layer for binary classification
#output = Dense(1, activation='sigmoid')(lstm_output)

# Define the model
word_model = Model(inputs=word_input, outputs=lstm_output, name='word_model')

# Compile the model
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
word_model.summary()

Model: "word_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding (Embedding)       (None, 50, 300)           933000    
                                                                 
 lstm (LSTM)                 (None, 16)                20288     
                                                                 
Total params: 953,288
Trainable params: 20,288
Non-trainable params: 933,000
_________________________________________________________________


In [ ]:
# Concatenate audio and text, time models
from tensorflow.keras.layers import Reshape, RepeatVector, Concatenate
# Reshape word_model output to match the shape of audio_model output
audio_model_output = layers.GlobalAveragePooling1D()(audio_model.output[1])
# Drop-out layer before the final Classification-Head
audio_model_output = layers.Dropout(0.5) (audio_model_output)

concatenated_output = Concatenate()([audio_model_output, word_model.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
audio_word= Dense(1, activation='sigmoid')(concatenated_output)
audio_word_model = Model(inputs=[audio_model.input, word_model.input], outputs=audio_word, name='combined_model')
audio_word_model.summary()

Model: "combined_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16000)]      0           []                               
                                                                                                  
 tf_wav2_vec2_model (TFWav2Vec2  TFWav2Vec2BaseModel  94371712   ['input_1[0][0]']                
 Model)                         Output(last_hidden_                                               
                                state=(None, 49, 76                                               
                                8),                                                               
                                 extract_features=(                                               
                                None, 49, 512),                                      

In [ ]:
from datasets import load_from_disk
final_combined_dataset=load_from_disk('/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/final_combined_data_original_augmented')
final_combined_dataset=final_combined_dataset.train_test_split(test_size=0.2)
final_trained_dataset=final_combined_dataset['train'].train_test_split(test_size=0.2)

In [ ]:
import numpy as np

In [ ]:
# Split the dataset into audio, text and time along with train, val, test sets
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Convert nested lists to numpy arrays
audio_train = pad_sequences(final_trained_dataset['train']['input_values'], maxlen=16000, dtype=np.float32, padding='post', truncating='post')
audio_train = np.asarray(audio_train).astype(np.float32)
audio_train = tf.convert_to_tensor(audio_train)
word_train = np.asarray(final_trained_dataset['train']['word']).astype(np.float32)
word_train = tf.convert_to_tensor(word_train)
audio_val = pad_sequences(final_trained_dataset['test']['input_values'], maxlen=16000, dtype=np.float32, padding='post', truncating='post')
audio_val = np.asarray(audio_val).astype(np.float32)
audio_val = tf.convert_to_tensor(audio_val)
audio_test = pad_sequences(final_combined_dataset['test']['input_values'], maxlen=16000, dtype=np.float32, padding='post', truncating='post')
audio_test = np.asarray(audio_test).astype(np.float32)
audio_test = tf.convert_to_tensor(audio_test)
word_val = np.asarray(final_trained_dataset['test']['word']).astype(np.float32)
word_val = tf.convert_to_tensor(word_val)
word_test = np.asarray(final_combined_dataset['test']['word']).astype(np.float32)
word_test = tf.convert_to_tensor(word_test)


In [ ]:
import numpy as np
y_train=np.asarray(final_trained_dataset['train']['label']).astype(np.float32)
y_train = tf.convert_to_tensor(y_train)
y_val=np.asarray(final_trained_dataset['test']['label']).astype(np.float32)
y_val = tf.convert_to_tensor(y_val)
y_test=np.asarray(final_combined_dataset['test']['label']).astype(np.float32)
y_test = tf.convert_to_tensor(y_test)

In [ ]:
# Compile and train the model
audio_word_model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy', tf.keras.metrics.Precision(),tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=10,
                                            restore_best_weights=True)
audio_word_model.fit([audio_train, word_train], y_train,
                       epochs=50, batch_size=16, validation_data=([audio_val, word_val], y_val),
                       callbacks=callback)

Epoch 1/50
2158/2158 [==============================] - 423s 189ms/step - loss: 0.6670 - accuracy: 0.5840 - precision: 0.5130 - recall: 0.3079 - auc: 0.5947 - val_loss: 0.6393 - val_accuracy: 0.6032 - val_precision: 0.5204 - val_recall: 0.5937 - val_auc: 0.6566
Epoch 2/50
2158/2158 [==============================] - 401s 186ms/step - loss: 0.6447 - accuracy: 0.6028 - precision: 0.5395 - recall: 0.4101 - auc: 0.6395 - val_loss: 0.6285 - val_accuracy: 0.6187 - val_precision: 0.5933 - val_recall: 0.2657 - val_auc: 0.6690
Epoch 3/50
2158/2158 [==============================] - 397s 184ms/step - loss: 0.6430 - accuracy: 0.6076 - precision: 0.5454 - recall: 0.4299 - auc: 0.6476 - val_loss: 0.6335 - val_accuracy: 0.6213 - val_precision: 0.5683 - val_recall: 0.3743 - val_auc: 0.6720
Epoch 4/50
2158/2158 [==============================] - 397s 184ms/step - loss: 0.6531 - accuracy: 0.6168 - precision: 0.5724 - recall: 0.3689 - auc: 0.6374 - val_loss: 0.6432 - val_accuracy: 0.6222 - val_precision

In [ ]:
# Evaluate the model
audio_word_model.evaluate([audio_test, word_test], y_test)

338/338 [==============================] - 64s 184ms/step - loss: 0.4184 - accuracy: 0.8019 - precision: 0.7551 - recall: 0.7748 - auc: 0.8852


[0.41843363642692566,
 0.8019095063209534,
 0.7551064491271973,
 0.7748048901557922,
 0.8852483034133911]